In [1]:
%load_ext autoreload
%autoreload 1

In [2]:
%aimport utils
%aimport Networks

In [3]:
import numpy as np
import pandas as pd
import os
import tensorflow as tf
from os import path
from utils import load_embeddings_and_ids, User
from Networks import VBPR_Network_Evaluation

In [4]:
# use a single GPU because we want to be nice with other people :)
os.environ["CUDA_VISIBLE_DEVICES"]="1"

###  Load pre-trained ResNet50 image embeddings

In [5]:
resnet50_embeddings,\
artwork_ids,\
artwork_id2index = load_embeddings_and_ids(
'/mnt/workspace/Ugallery/ResNet50/', 'flatten_1.npy', 'ids')

In [6]:
n_artworks = len(artwork_ids)
n_artworks

13297

### Load user list from transactions

In [7]:
sales_df = pd.read_csv('./valid_sales.csv')
user_ids = sales_df.customer_id.unique()
n_users = len(user_ids)

In [8]:
n_users

2919

### Load saved VBPR network and compute tensors

In [9]:
MODEL_PATH = '/mnt/workspace/pamessina_models/ugallery/VBPR/v3_hidinglast/'

In [10]:
all_indexes = list(range(n_artworks))

In [11]:
with tf.Graph().as_default():
    network = VBPR_Network_Evaluation(
        n_users=n_users,
        n_items=n_artworks,
        user_latent_dim=128,
        item_latent_dim=64,
        item_visual_dim=64,
        pretrained_dim=2048,
    )
    with tf.Session() as sess:
        saver = tf.train.Saver()
        saver.restore(sess, tf.train.latest_checkpoint(MODEL_PATH))
        user_latent_vectors = network.get_user_latent_vectors(sess)
        item_vectors, item_biases = network.get_item_final_vector_bias(sess, resnet50_embeddings, all_indexes)

INFO:tensorflow:Restoring parameters from /mnt/workspace/pamessina_models/ugallery/VBPR/v3_hidinglast/


In [12]:
item_vectors.shape, item_biases.shape

((13297, 128), (13297,))

In [13]:
user_latent_vectors.shape

(2919, 128)

In [14]:
dir_path = "/mnt/workspace/Ugallery/VBPR/v3_hidinglast/"
os.makedirs(dir_path, exist_ok=True)

# users
user_latent_vectors.dump(dir_path + "user_vectors.npy")
with open(dir_path + 'user_ids', 'w') as f:
    for _id in user_ids:
        f.write('%d\n' % _id)
        
# items
item_vectors.dump(dir_path + "item_vectors.npy")
item_biases.dump(dir_path + "item_biases.npy")
with open(dir_path + 'items_ids', 'w') as f:
    for _id in artwork_ids:
        f.write('%d\n' % _id)